<a href="https://colab.research.google.com/github/ghofranetayari/Social-Sentiment-Detection-with-Machine-Learning/blob/main/Twitter_Sentiment_Analysis_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
! pip install kaggle

In [11]:
# configure the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [12]:
# API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 88% 71.0M/80.9M [00:00<00:00, 167MB/s]
100% 80.9M/80.9M [00:00<00:00, 167MB/s]


In [13]:
# extracting the compresse dataset

from zipfile import ZipFile
dataset ='/content/sentiment140.zip'
#open path and extracting
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')


The dataset is extracted


Importing the dependencies

In [14]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data processing

In [17]:
# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' , encoding = 'ISO-8859-1')


In [18]:
# checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [19]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [20]:
# naming the columns and reading the dataset again

column_names = ['target' , 'id' , 'date' , 'flag' , 'user' , 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' , names=column_names , encoding = 'ISO-8859-1')


In [21]:
# checking the number of rows and columns
twitter_data.shape

(1600000, 6)

In [22]:
# printing the first 5 rows of the dataframe
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [26]:
# counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [27]:
# checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Convert the target "4" to "1"

In [28]:
twitter_data.replace({'target' :{4:1}},inplace=True)

In [29]:
# checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 -->Negative Tweet
1-->Positive Tweet

**Stemming**

****
Stemming is the process of reducing a word to its Root word


example : actor actress ,acting = act

In [30]:
port_stem = PorterStemmer()

In [31]:
def stemming (content):
    # Nettoie le contenu en supprimant tout caractère non alphabétique
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
      # Convertit le texte en minuscules
  stemmed_content = stemmed_content.lower()
      # Divise le texte en mots
  stemmed_content = stemmed_content.split()
      # Applique le stemming aux mots non présents dans la liste des mots d'arrêt
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]

  return stemmed_content

In [32]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming) #50min

In [33]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[switchfoot, http, twitpic, com, zl, awww, bum..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"[upset, updat, facebook, text, might, cri, res..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"[kenichan, dive, mani, time, ball, manag, save..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[whole, bodi, feel, itchi, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","[nationwideclass, behav, mad, see]"


In [34]:
print(twitter_data['stemmed_content'])

0          [switchfoot, http, twitpic, com, zl, awww, bum...
1          [upset, updat, facebook, text, might, cri, res...
2          [kenichan, dive, mani, time, ball, manag, save...
3                     [whole, bodi, feel, itchi, like, fire]
4                         [nationwideclass, behav, mad, see]
                                 ...                        
1599995                     [woke, school, best, feel, ever]
1599996    [thewdb, com, cool, hear, old, walt, interview...
1599997                   [readi, mojo, makeov, ask, detail]
1599998    [happi, th, birthday, boo, alll, time, tupac, ...
1599999    [happi, charitytuesday, thenspcc, sparkschar, ...
Name: stemmed_content, Length: 1600000, dtype: object


In [35]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [59]:

# seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [80]:
print(X)

In [78]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data to training data and test data

In [44]:
X_train, X_test, Y_train, Y_train = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [45]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [46]:
print(X_train)

[list(['watch', 'saw', 'iv', 'drink', 'lil', 'wine'])
 list(['hatermagazin'])
 list(['even', 'though', 'favourit', 'drink', 'think', 'vodka', 'coke', 'wipe', 'mind', 'time', 'think', 'im', 'gonna', 'find', 'new', 'drink'])
 ... list(['eager', 'monday', 'afternoon'])
 list(['hope', 'everyon', 'mother', 'great', 'day', 'wait', 'hear', 'guy', 'store', 'tomorrow'])
 list(['love', 'wake', 'folger', 'bad', 'voic', 'deeper'])]


In [42]:
print(X_test)

[list(['mmangen', 'fine', 'much', 'time', 'chat', 'twitter', 'hubbi', 'back', 'summer', 'amp', 'tend', 'domin', 'free', 'time'])
 list(['ah', 'may', 'show', 'w', 'ruth', 'kim', 'amp', 'geoffrey', 'sanhueza'])
 list(['ishatara', 'mayb', 'bay', 'area', 'thang', 'dammit']) ...
 list(['destini', 'nevertheless', 'hooray', 'member', 'wonder', 'safe', 'trip'])
 list(['feel', 'well']) list(['supersandro', 'thank'])]


In [81]:
# converting the textual data to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
print(X_train)

In [ ]:
print(X_test)

Training the Machine Learning Model

Logistic Regression

In [82]:
model = LogidcticRegression(max_iter=1000)

NameError: name 'LogidcticRegression' is not defined

In [83]:
model.fit(X_train, Y_train)

NameError: name 'model' is not defined

Model Evaluation

Accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accurarcy_score(Y_train, X_train_prediction)

In [ ]:
print('Accuracy score on the training data : ', training_data_accuracy)

In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accurarcy_score(Y_test, X_test_prediction)

In [ ]:
print('Accuracy score on the training data : ', test_data_accuracy)

Model accuracy =77.8%

In [ ]:
filname = 'trained_model.sav'
pickle.dump(model, open(filename,'wb'))

Using the saved model for future predictions

In [ ]:
# loading th saved model
loaded_model = pickle.load(open(/content/trained_model.sav' , 'rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')

  else:
    print('Positive Tweet')